In [7]:
import pandas as pd


clustering_algorithms = ['gaussian_mixture', 'kmeans']
fusion_types = ["add", "cat"]
weights = ["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"]
iteration = 10
class_num = ["23", "34"]
conv_types = ["se", "re"]
eval_methods = ['ari', 'nmi', 'purity']

name_column = []
for c_num in class_num:
    data_name = "rico_" + c_num
    name_column.append(data_name)
    row_name = 'seq2seq_' + c_num
    name_column.append(row_name)
    row_name = 'seq2seq_tree_' + c_num
    name_column.append(row_name)
    for c_type in conv_types:
        row_name = 'conv_' + c_type + '_' + c_num
        name_column.append(row_name)
    for f_type in fusion_types:
        row_name = 'rico_' + c_num + '_seq2seq_' + c_num + '_' + f_type
        name_column.append(row_name)
    for f_type in fusion_types:
        row_name = 'rico_' + c_num + '_seq2seq_tree_' + c_num + '_' + f_type
        name_column.append(row_name)
    for f_type in fusion_types:
        row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type
        name_column.append(row_name)
    for c_type in conv_types:
        for f_type in fusion_types:
            row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
            name_column.append(row_name)
        for f_type in fusion_types:
            row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
            name_column.append(row_name)
        for f_type in fusion_types:
            row_name = "seq2seq_tree_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
            name_column.append(row_name)
    for f_type in fusion_types:
        for w in weights:
            row_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + f_type + "_" + w
            name_column.append(row_name)
    for f_type in fusion_types:
        for w in weights:
            row_name = "rico_" + c_num + "_seq2seq_tree_" + c_num + "_" + f_type + "_" + w
            name_column.append(row_name)
    for f_type in fusion_types:
        for w in weights:
            row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type + "_" + w
            name_column.append(row_name)
    for c_type in conv_types:
        for w in weights:
            for f_type in fusion_types:
                row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w             
                name_column.append(row_name)
            for f_type in fusion_types:
                row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                name_column.append(row_name)
            for f_type in fusion_types:
                row_name = "seq2seq_tree_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                name_column.append(row_name)

column_name = [str(x) for x in range(10)]
df_ari_gmm = pd.DataFrame(columns=column_name)
df_ari_gmm = df_ari_gmm.assign(name = name_column)
df_ari_gmm.set_index('name', inplace=True)
df_ari_km = pd.DataFrame(columns=column_name)
df_ari_km = df_ari_km.assign(name = name_column)
df_ari_km.set_index('name', inplace=True)

df_nmi_gmm = pd.DataFrame(columns=column_name)
df_nmi_gmm = df_ari_gmm.assign(name = name_column)
df_nmi_gmm.set_index('name', inplace=True)
df_nmi_km = pd.DataFrame(columns=column_name)
df_nmi_km = df_ari_km.assign(name = name_column)
df_nmi_km.set_index('name', inplace=True)

df_purity_gmm = pd.DataFrame(columns=column_name)
df_purity_gmm = df_ari_gmm.assign(name = name_column)
df_purity_gmm.set_index('name', inplace=True)
df_purity_km = pd.DataFrame(columns=column_name)
df_purity_km = df_ari_km.assign(name = name_column)
df_purity_km.set_index('name', inplace=True)

def run(data_name, iter, row_name):    
    lres = []
    for i, eval_method in enumerate(eval_methods):
        csv_path = 'csv_tree/' + eval_method + '/' + data_name + ".csv"
        with open(csv_path, 'r') as fd:
            res = fd.read()
        lres = res.split('\n')
        if eval_method == 'purity':
            temp = lres[-2].split('Total,')[-1].split(',')
            df_purity_gmm.loc[row_name, str(iter)] = float(temp[0].strip('\%'))/100
            df_purity_km.loc[row_name, str(iter)] = float(temp[1].strip('\%'))/100
        elif eval_method == 'nmi':
            temp = lres[-2].split(',')
            df_nmi_gmm.loc[row_name, str(iter)] = float(temp[0])
            df_nmi_km.loc[row_name, str(iter)] = float(temp[1])
        else:
            temp = lres[-2].split(',')
            df_ari_gmm.loc[row_name, str(iter)] = float(temp[0])
            df_ari_km.loc[row_name, str(iter)] = float(temp[1])

for c_num in class_num:
    data_name = "rico_" + c_num
    run(data_name, 0, data_name)
    for _iter in range(iteration):
        data_name = "seq2seq_" + c_num + "_" + str(_iter)
        row_name = 'seq2seq_' + c_num
        run(data_name, _iter, row_name)
        data_name = "seq2seq_tree_" + c_num + "_" + str(_iter)
        row_name = 'seq2seq_tree_' + c_num
        run(data_name, _iter, row_name)
        for c_type in conv_types:
            data_name = "conv_" + c_type + "_" + c_num + "_" + str(_iter)
            row_name = 'conv_' + c_type + '_' + c_num
            run(data_name, _iter, row_name)
        for f_type in fusion_types:
            data_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + str(_iter) + "_" + f_type
            row_name = 'rico_' + c_num + '_seq2seq_' + c_num + '_' + f_type
            run(data_name, _iter, row_name)
        for f_type in fusion_types:
            data_name = "rico_" + c_num + "_seq2seq_tree_" + c_num + "_" + str(_iter) + "_" + f_type
            row_name = 'rico_' + c_num + '_seq2seq_tree_' + c_num + '_' + f_type
            run(data_name, _iter, row_name)
        for f_type in fusion_types:
            data_name = "conv_re_" + c_num + "_" + str(_iter) + "_conv_se_" + c_num + "_" + str(_iter) + "_" + f_type
            row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type
            run(data_name, _iter, row_name)
        for c_type in conv_types:
            for f_type in fusion_types:
                data_name = ("rico_" + c_num + "_conv_" + c_type + "_" +
                                c_num + "_" + str(_iter) + "_" + f_type)
                row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
                run(data_name, _iter, row_name)
            for f_type in fusion_types:
                data_name = ("seq2seq_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                c_num + "_" + str(_iter) + "_" + f_type)
                row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
                run(data_name, _iter, row_name)
            for f_type in fusion_types:
                data_name = ("seq2seq_tree_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                c_num + "_" + str(_iter) + "_" + f_type)
                row_name = "seq2seq_tree_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type
                run(data_name, _iter, row_name)
        for f_type in fusion_types:
            for w in weights:
                data_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + str(_iter) + "_" + f_type + "_" + w
                row_name = "rico_" + c_num + "_seq2seq_" + c_num + "_" + f_type + "_" + w
                run(data_name, _iter, row_name)
        for f_type in fusion_types:
            for w in weights:
                data_name = "rico_" + c_num + "_seq2seq_tree_" + c_num + "_" + str(_iter) + "_" + f_type + "_" + w
                row_name = "rico_" + c_num + "_seq2seq_tree_" + c_num + "_" + f_type + "_" + w
                run(data_name, _iter, row_name)
        for f_type in fusion_types:
            for w in weights:
                data_name = "conv_re_" + c_num + "_" + str(_iter) + "_conv_se_" + c_num + "_" + str(_iter) + "_" + f_type + "_" + w
                row_name = "conv_re_" + c_num + "_conv_se_" + c_num + "_" + f_type + "_" + w
                run(data_name, _iter, row_name)
        for c_type in conv_types:
            for w in weights:
                for f_type in fusion_types:
                    data_name = ("rico_" + c_num + "_conv_" + c_type + "_" +
                                    c_num + "_" + str(_iter) + "_" + f_type + "_" + w)
                    row_name = "rico_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w             
                    run(data_name, _iter, row_name)
                for f_type in fusion_types:
                    data_name = ("seq2seq_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                    c_num + "_" + str(_iter) + "_" + f_type + "_" + w)
                    row_name = "seq2seq_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                    run(data_name, _iter, row_name)
                for f_type in fusion_types:
                    data_name = ("seq2seq_tree_" + c_num + "_" + str(_iter) + "_conv_" + c_type + "_" +
                                    c_num + "_" + str(_iter) + "_" + f_type + "_" + w)
                    row_name = "seq2seq_tree_" + c_num + "_conv_" + c_type + "_" + c_num + "_" + f_type + "_" + w
                    run(data_name, _iter, row_name)


In [8]:
df_ari_gmm['Mean'] = df_ari_gmm.mean(axis=1)
df_ari_gmm['Std'] = df_ari_gmm.std(axis=1)
df_ari_km['Mean'] = df_ari_km.mean(axis=1)
df_ari_km['Std'] = df_ari_km.std(axis=1)
df_nmi_gmm['Mean'] = df_nmi_gmm.mean(axis=1)
df_nmi_gmm['Std'] = df_nmi_gmm.std(axis=1)
df_nmi_km['Mean'] = df_nmi_km.mean(axis=1)
df_nmi_km['Std'] = df_nmi_km.std(axis=1)
df_purity_gmm['Mean'] = df_purity_gmm.mean(axis=1)
df_purity_gmm['Std'] = df_purity_gmm.std(axis=1)
df_purity_km['Mean'] = df_purity_km.mean(axis=1)
df_purity_km['Std'] = df_purity_km.std(axis=1)

In [9]:
df_nmi_km

,0,1,2,3,4,5,6,7,8,9,Mean,Std
name,,,,,,,,,,,,
rico_23,0.535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5350,0.000000
seq2seq_23,0.549,0.532,0.548,0.559,0.539,0.519,0.544,0.514,0.563,0.572,0.5439,0.017621
seq2seq_tree_23,0.638,0.644,0.635,0.622,0.63,0.646,0.638,0.645,0.635,0.625,0.6358,0.007795
conv_se_23,0.611,0.609,0.581,0.549,0.593,0.607,0.6,0.596,0.6,0.608,0.5954,0.017693
conv_re_23,0.507,0.435,0.469,0.472,0.461,0.468,0.46,0.473,0.449,0.484,0.4678,0.018389
...,...,...,...,...,...,...,...,...,...,...,...,...
rico_34_conv_re_34_cat_0.9,0.552,0.475,0.525,0.527,0.521,0.517,0.532,0.52,0.528,0.543,0.5240,0.019261
seq2seq_34_conv_re_34_add_0.9,0.559,0.489,0.527,0.546,0.549,0.514,0.54,0.541,0.529,0.521,0.5315,0.019195
seq2seq_34_conv_re_34_cat_0.9,0.548,0.498,0.536,0.516,0.544,0.508,0.527,0.531,0.528,0.549,0.5285,0.016163


In [10]:
df_ari_gmm.to_csv('../result/ari_gmm_tree.csv')
df_ari_km.to_csv('../result/ari_km_tree.csv')
df_nmi_gmm.to_csv('../result/nmi_gmm_tree.csv')
df_nmi_km.to_csv('../result/nmi_km_tree.csv')
df_purity_gmm.to_csv('../result/purity_gmm_tree.csv')
df_purity_km.to_csv('../result/purity_km_tree.csv')
